#### 1.) 패키지 업로드 

In [1]:
import pandas as pd
#pd.set_option('display.max_columns', None)
import numpy as np
from numpy import array
import datetime
from tqdm import tqdm
import scipy
from scipy import stats
from scipy.stats import skew,kurtosis,describe,variation
from pymongo import MongoClient

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
def _centered(arr, newsize):
    # Return the center newsize portion of the array.
    newsize = np.asarray(newsize)
    currsize = np.array(arr.shape)
    startind = (currsize - newsize) // 2
    endind = startind + newsize
    myslice = [slice(startind[k], endind[k]) for k in range(len(endind))]
    return arr[tuple(myslice)]

scipy.signal.signaltools._centered = _centered # Scipy Tool 조정 

#### 2.) 데이터 연결 & 업로드 

In [3]:
def get_mongo_data(db_name,collection_name,condition = False):
    server_address = "private_server"
    client_update = MongoClient("mongodb://private_company_client@{0}/admin".format(server_address))
    db = client_update[db_name]
    col = db[collection_name]
    print = condition
    if condition is not False:
        df = pd.DataFrame(list(col.find({"serialNumber":{"$in":condition}})))
    else:
        df = pd.DataFrame(list(col.find({})))
    return df

In [4]:
data = get_mongo_data('privateDB','injectionmoldData') # Injection Mold 
plc = get_mongo_data('privateDB','neonentplcdata')     # PLC 

#### 3.) 개별 사출기 데이터 준비 
- Ex. 사출기 18호기 

In [5]:
data = data
plc = plc
machine_number = 18
matched = 'yes'

In [6]:
def data_select(data,plc,machine_number):
    data = data[data['PlantNo']=='A01-0'+str(machine_number)]
    plc = plc[plc['LineName']== str(machine_number)]
    return data, plc

In [7]:
def Adjusted_Data(data,plc,machine_number):
    
    data, plc = data_select(data,plc,machine_number) # Ex.) 사출기 18호 데이터 출력 
    data = data[(data.CreateDate < plc.CreateDate.iloc[-1])] # Injection Mold Data 날짜 조정 --> PLC 데이터의 마지막 날짜 이후의 값 제거 [ 데이터 매칭 시간 감소 목적]
    data = data.reset_index().set_index('CreateDate').loc['2023-01-18 00:00:00':'2023.06.19 23:35:08.437'] # 2023-01-18 00:00:00 기준으로 PLC 수집 방식 변화 
    data = data.reset_index().set_index('index')
    plc = plc.reset_index().set_index('CreateDate').loc['2023-01-18 00:00:00':'2023.05.30 06:17:05.990']   # 2023-01-18 00:00:00 기준으로 PLC 수집 방식 변화 
    plc = plc.reset_index().set_index('index')
    return data , plc # 분석에 활용한 데이터에 해당하는 날짜 선택 
data, plc = Adjusted_Data(data,plc,machine_number) # 날짜 조정 & 매칭 준비 완료 

In [8]:
data.CreateDate

index
7510     2023-01-18 23:31:48.679
7513     2023-01-18 23:32:33.926
7518     2023-01-18 23:34:33.923
7524     2023-01-18 23:35:53.934
7529     2023-01-18 23:36:43.927
                   ...          
420428   2023-06-14 10:00:17.279
420429   2023-06-14 10:01:27.290
420430   2023-06-14 10:02:42.379
420431   2023-06-14 10:03:52.354
420432   2023-06-14 10:05:02.298
Name: CreateDate, Length: 34906, dtype: datetime64[ns]

In [9]:
plc.CreateDate

index
2023410    2023-02-07 01:06:00.584
2023416    2023-02-07 01:06:01.622
2023422    2023-02-07 01:06:02.649
2023428    2023-02-07 01:06:03.691
2023434    2023-02-07 01:06:04.720
                     ...          
18327830   2023-06-14 13:00:07.525
18327832   2023-06-14 13:00:08.569
18327834   2023-06-14 13:00:09.601
18327836   2023-06-14 13:00:10.620
18327838   2023-06-14 13:00:11.682
Name: CreateDate, Length: 2707859, dtype: datetime64[ns]

#### 4.) 데이터 전처리 
- 사출 & PLC 시계열 데이터 매칭 

In [10]:
def preprocessing(data,plc,machine_number,matched):

    #### 1.) 사출 생산 사이에 발생한 PLC 시계열 데이터들의 통계량 계산 이후 매칭 진행 
    if matched =='yes':
        i = 0
        index_plc = []
        index_data = []
        matched_df = pd.DataFrame()
        total_df = pd.DataFrame()
        for n, i in tqdm(enumerate(range(len(data)))):
            length = 0
            if machine_number == machine_number:  # 사출기별 사출데이터 생산 시간 고려
                matched_plc = plc[(plc.CreateDate <= data.CreateDate.iloc[i]) & (plc.CreateDate > data.CreateDate.iloc[i] - datetime.timedelta(seconds=time))] 
                length = len(matched_plc)
                if length != 0:
                    total_df = pd.concat([total_df, data.iloc[[i]]]) 
                    matched_df = pd.concat([matched_df, matched_plc]) 
                    index_data.append(n)
                    for k in range(length):
                        index_plc.append(n) # 교집합 인덱스 
        print('Matched Data Exists')
        print('Used Matched Injection Mold & PLC Data for Prediction')
        matched_df['index1'] = index_plc            
        total_df['PassOrFail'] = total_df['PassOrFail'].replace({"Y":0, "N":1})
        total_df['index1'] = index_data  
        total = pd.merge(total_df,matched_df, how='left',on = 'index1') 

        for col in total.columns:
            try : total[col] = total[col].astype('float')
            except: pass     
        total = total.rename(columns = {'Value1' : 'WaxTank_Temperature','Value2' : 'Injection_Temperature','Value3' : 'CirculationHose_Temperature','Value4' : 'Nozzle_Temperature','InjectionTemperature':'PLC_InjectionTemperature','NozzleTemperature':'PLC_NozzleTemperature','CoolingTime':'PLC_CoolingTime','InjectionPressure':'PLC_InjectionPressure','MultiTankTemperature':'PLC_MultiTankTemperature','CirculationTank':'PLC_CirculationTank'}) 
        total = total.rename(columns = {'_id_x' :'PLC_id','CreateDate_x' :'PLC_CreateDate','_id_y' :'_id','CreateDate_y' :'CreateDate'})
        total = total[['YMdate','CreateDate','_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','Aqty','WaxTank_Temperature','Injection_Temperature','CirculationHose_Temperature','Nozzle_Temperature','PLC_CreateDate','PLC_id','StartAddress','LineName','StartEnd','Count','PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','index1','PassOrFail']]
        total['YMdate'] = total['YMdate'].str.replace('오전', 'AM')
        total['YMdate'] = total['YMdate'].str.replace('오후', 'PM')
        df = total['YMdate'].str.split(" ",expand=True).rename(columns = {0 : 'Date',1 : 'AM/PM',2:'Time'})
        total['YMdate'] = pd.to_datetime(df['Date'] + ' ' + df['Time']  + ' ' + df['AM/PM'])

        #### 2.) 사출기별 PLC데이터 요약통계량 기반 파생변수 Custom 계산 
        if machine_number == 13:  
             ####.....
        elif machine_number == 14:
             ####.....
        elif machine_number == 15:
            ####.....
        elif machine_number == 16: 
            ####..... 
        elif machine_number == 17:
            ####.....
        elif machine_number == 18:
            df_mean = total.groupby('index1').mean()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank','PassOrFail']]
            df_mean = df_mean.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Mean','PLC_InjectionPressure' : 'PLC_InjectionPressure_Mean','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Mean','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Mean','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Mean','PLC_CirculationTank' : 'PLC_CirculationTank_Mean'})
            df_median = total.groupby('index1').median()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_median = df_median.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Median','PLC_InjectionPressure' : 'PLC_InjectionPressure_Median','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Median','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Median','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Median','PLC_CirculationTank' : 'PLC_CirculationTank_Median'})
            df_min = total.groupby('index1').min()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_min = df_min.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Min','PLC_InjectionPressure' : 'PLC_InjectionPressure_Min','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Min','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Min','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Min','PLC_CirculationTank' : 'PLC_CirculationTank_Min'})
            df_max = total.groupby('index1').max()[['PLC_CoolingTime','PLC_InjectionPressure','PLC_MultiTankTemperature','PLC_InjectionTemperature','PLC_NozzleTemperature','PLC_CirculationTank']]
            df_max = df_max.rename(columns = {'PLC_CoolingTime' : 'PLC_CoolingTime_Max','PLC_InjectionPressure' : 'PLC_InjectionPressure_Max','PLC_MultiTankTemperature' : 'PLC_MultiTankTemperature_Max','PLC_InjectionTemperature' : 'PLC_InjectionTemperature_Max','PLC_NozzleTemperature' : 'PLC_NozzleTemperature_Max','PLC_CirculationTank' : 'PLC_CirculationTank_Max'})
            df_final = pd.concat([df_mean,df_median,df_min,df_max],axis=1)

        Final_Injection_Mold = total_df
        for col in Final_Injection_Mold.columns:
            try : Final_Injection_Mold[col] = Final_Injection_Mold[col].astype('float')
            except: pass
        Final_Injection_Mold = Final_Injection_Mold.rename(columns = {'Value1' : 'WaxTank_Temperature','Value2' : 'Injection_Temperature','Value3' : 'CirculationHose_Temperature','Value4' : 'Nozzle_Temperature'})
        Final_Matched =  df_final.drop(['PassOrFail'],axis=1)  
        Final_Injection_Mold = Final_Injection_Mold.set_index('index1')
        Final_Injection_Mold = Final_Injection_Mold.loc[Final_Matched.index]
        Final = pd.concat([Final_Injection_Mold,Final_Matched ],axis=1)
        return Final
    
    
    else :
        #### 3.) 수집 오류 등의 문제로 사출 + PLC 매칭 불가능할 경우 사출데이터만 사용하도록 예외처리 
        print('Use Original Injection Mold Data [ Nonmatched with PLC ]')
        for col in data.columns:
            try : data[col] = data[col].astype('float')
            except: pass  
        #### 4.) 현재 각 1개 행마다 사출데이터들 입력값들의 통계량 기반 파생변수 Column 다수 생성 시도         
        data = data.rename(columns = {'Value1' : 'WaxTank_Temperature','Value2' : 'Injection_Temperature','Value3' : 'CirculationHose_Temperature','Value4' : 'Nozzle_Temperature'})
        data['PassOrFail'] = data['PassOrFail'].replace({"Y":0, "N":1})
        Min = data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).min(axis='columns') #Binary_df
        Mean = data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).mean(axis='columns')
        Quantile_1=  data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).quantile(0.25,axis='columns')
        Quantile_40=  data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).quantile(0.4,axis='columns')
        Median = data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).quantile(0.5,axis='columns')
        Quantile_60=  data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).quantile(0.6,axis='columns')
        Quantile_3 =  data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).quantile(0.75,axis='columns')
        Max =  data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).max(axis='columns')
        Skew =  data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).skew(axis='columns')
        Kurtosis =  data.drop(['_id','UniqeNum','PlantNo','PlantName','ItmeCode','ItemName','YMdate','CreateDate','PassOrFail','FaultyType','Aqty'],axis=1).kurtosis(axis='columns')
        Descriptives = pd.concat([Min,Mean,Quantile_1,Median,Quantile_3,Max,Skew,Kurtosis,Quantile_40,Quantile_60],axis=1) #Skew,Kurtosis
        Descriptives = Descriptives.rename(columns={0.00:"Min",1.00:"Mean",2.00:"Max",0.25:"Quantile_1",0.50:"Median",0.75:"Quantile_3",3.00:'Skew',4.00:'Kurtosis',0.40:'Quantile_40',0.60:'Quantile_60'})
        Final = pd.concat([data,Descriptives],axis=1)
        return Final

#### 5.) 학습데이터 저장 

In [11]:
data = data
plc = plc
machine_number = 18 # 13,14,15,16,17,18
matched = 'yes' # 'yes', no'

time = data.CreateDate.diff().mode().dt.seconds[0].tolist()
print('사출기 생산시간:',time,'seconds') 

Final_Matched = preprocessing(data,plc,machine_number,matched = 'yes')
Final_NonMatched = preprocessing(data,plc,machine_number,matched = 'no')

사출기 생산시간: 60 seconds


34906it [14:41, 39.59it/s]


Matched Data Exists
Used Matched Injection Mold & PLC Data for Prediction
Use Original Injection Mold Data [ Nonmatched with PLC ]


In [12]:
Final_Matched

,CreateDate,_id,UniqeNum,PlantNo,PlantName,ItmeCode,ItemName,YMdate,WaxTank_Temperature,Injection_Temperature,CirculationHose_Temperature,Nozzle_Temperature,Aqty,PassOrFail,FaultyType,PLC_CoolingTime_Mean,PLC_InjectionPressure_Mean,PLC_MultiTankTemperature_Mean,PLC_InjectionTemperature_Mean,PLC_NozzleTemperature_Mean,PLC_CirculationTank_Mean,PLC_CoolingTime_Median,PLC_InjectionPressure_Median,PLC_MultiTankTemperature_Median,PLC_InjectionTemperature_Median,PLC_NozzleTemperature_Median,PLC_CirculationTank_Median,PLC_CoolingTime_Min,PLC_InjectionPressure_Min,PLC_MultiTankTemperature_Min,PLC_InjectionTemperature_Min,PLC_NozzleTemperature_Min,PLC_CirculationTank_Min,PLC_CoolingTime_Max,PLC_InjectionPressure_Max,PLC_MultiTankTemperature_Max,PLC_InjectionTemperature_Max,PLC_NozzleTemperature_Max,PLC_CirculationTank_Max
index1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
17557.0,2023-03-16 04:18:24.270,646f1339057c35ebeea3a472,2.303161e+14,A01-018,사출기(18호),N1555-7SH,NE-EV BRKT-CAM 사출품[SCR420],2023-03-16 오후 1:18:19,60.5,63.3,56.4,58.9,920.0,0.0,NaN,46.212121,11.969697,60.448485,63.269697,56.400000,58.951515,50.0,22.0,60.5,63.20,56.4,58.9,4.0,0.0,60.3,62.7,56.3,58.9,50.0,22.0,60.5,63.9,56.5,59.1
17558.0,2023-03-16 04:19:19.259,646f1339057c35ebeea3a47c,2.303161e+14,A01-018,사출기(18호),N1555-7SH,NE-EV BRKT-CAM 사출품[SCR420],2023-03-16 오후 1:19:19,60.4,63.1,56.2,58.7,920.0,0.0,NaN,47.637931,8.982759,60.475862,63.113793,56.329310,58.872414,50.0,0.0,60.5,63.10,56.3,58.9,2.0,0.0,60.3,62.8,56.2,58.7,50.0,22.0,60.5,63.4,56.4,59.1
17559.0,2023-03-16 04:20:29.258,646f1339057c35ebeea3a48b,2.303161e+14,A01-018,사출기(18호),N1555-7SH,NE-EV BRKT-CAM 사출품[SCR420],2023-03-16 오후 1:20:26,60.5,62.8,56.2,58.7,928.0,1.0,기타,48.258621,12.758621,60.470690,62.981034,56.224138,58.848276,50.0,21.0,60.5,62.95,56.2,58.9,9.0,0.0,60.4,62.4,56.2,58.7,50.0,31.0,60.6,63.6,56.3,59.0
17560.0,2023-03-16 04:21:29.263,646f1339057c35ebeea3a496,2.303161e+14,A01-018,사출기(18호),N1555-7SH,NE-EV BRKT-CAM 사출품[SCR420],2023-03-16 오후 1:21:27,60.5,62.8,56.2,59.0,932.0,0.0,NaN,48.241379,12.482759,60.453448,62.806897,56.172414,58.820690,50.0,21.0,60.5,62.80,56.2,58.8,9.0,0.0,60.4,62.7,55.9,58.7,50.0,30.0,60.6,62.9,56.2,59.0
17561.0,2023-03-16 04:22:44.261,646f1339057c35ebeea3a4a7,2.303161e+14,A01-018,사출기(18호),N1555-7SH,NE-EV BRKT-CAM 사출품[SCR420],2023-03-16 오후 1:22:43,60.4,64.0,56.2,58.8,932.0,1.0,기포,48.034483,11.844828,60.431034,63.950000,56.053448,58.867241,50.0,21.0,60.4,63.85,56.1,58.9,6.0,0.0,60.3,61.5,55.9,58.7,50.0,41.0,60.5,67.0,56.2,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34901.0,2023-06-14 10:00:17.279,64898fb2851b289853a03fbd,2.306142e+14,A01-018,사출기(18호),N1217-8SH,M6AR1 BODY DETEND 사출품[SCR420],2023-06-14 오후 7:00:16,60.4,60.9,57.4,56.0,1030.0,0.0,NaN,29.224138,10.275862,60.468966,61.406897,57.391379,55.948276,30.0,0.0,60.5,61.30,57.4,55.9,5.0,0.0,60.3,60.9,57.2,55.9,30.0,24.0,60.7,62.3,57.5,56.0
34902.0,2023-06-14 10:01:27.290,64898ff7851b289853a03fbe,2.306142e+14,A01-018,사출기(18호),N1217-8SH,M6AR1 BODY DETEND 사출품[SCR420],2023-06-14 오후 7:01:24,60.5,61.6,57.4,56.0,1032.0,0.0,NaN,30.000000,0.000000,60.533333,61.200000,57.450877,55.978947,30.0,0.0,60.5,61.20,57.5,56.0,30.0,0.0,60.3,60.4,57.4,55.9,30.0,0.0,61.0,62.1,57.5,56.1
34903.0,2023-06-14 10:02:42.379,64899043851b289853a03fbf,2.306142e+14,A01-018,사출기(18호),N1217-8SH,M6AR1 BODY DETEND 사출품[SCR420],2023-06-14 오후 7:02:39,60.5,61.3,57.5,56.0,1034.0,0.0,NaN,30.000000,0.000000,60.494828,61.136207,57.444828,56.001724,30.0,0.0,60.5,61.30,57.4,56.0,30.0,0.0,60.4,60.4,57.4,55.9,30.0,0.0,60.6,61.6,57.5,56.1


In [14]:
Final_NonMatched

,CreateDate,_id,UniqeNum,PlantNo,PlantName,ItmeCode,ItemName,YMdate,WaxTank_Temperature,Injection_Temperature,CirculationHose_Temperature,Nozzle_Temperature,Aqty,PassOrFail,FaultyType,Min,Mean,Quantile_1,Median,Quantile_3,Max,Skew,Kurtosis,Quantile_40,Quantile_60
index,,,,,,,,,,,,,,,,,,,,,,,,,
7510,2023-01-18 23:31:48.679,646f045c53130a08d1d40123,2.301191e+14,A01-018,사출기(18호),N1570-7SH,신M6CF1 FINGER CONTROL 사출품(신형)[FCD600],2023-01-19 오전 8:31:38,60.1,63.0,57.1,59.1,8.0,0,NaN,57.1,59.825,58.600,59.60,60.825,63.0,0.513005,0.868686,59.30,59.90
7513,2023-01-18 23:32:33.926,646f045c53130a08d1d40129,2.301191e+14,A01-018,사출기(18호),N1570-7SH,신M6CF1 FINGER CONTROL 사출품(신형)[FCD600],2023-01-19 오전 8:32:29,60.1,62.9,56.5,58.8,10.0,1,기포,56.5,59.575,58.225,59.45,60.800,62.9,0.257877,0.408663,59.06,59.84
7518,2023-01-18 23:34:33.923,646f045c53130a08d1d40133,2.301191e+14,A01-018,사출기(18호),N1570-7SH,신M6CF1 FINGER CONTROL 사출품(신형)[FCD600],2023-01-19 오전 8:34:31,60.2,62.2,55.6,58.4,12.0,1,미성형,55.6,59.100,57.700,59.30,60.700,62.2,-0.366412,-0.306139,58.76,59.84
7524,2023-01-18 23:35:53.934,646f045c53130a08d1d4013f,2.301191e+14,A01-018,사출기(18호),N1570-7SH,신M6CF1 FINGER CONTROL 사출품(신형)[FCD600],2023-01-19 오전 8:35:51,60.5,62.3,55.3,58.0,14.0,0,NaN,55.3,59.025,57.325,59.25,60.950,62.3,-0.340478,-1.358159,58.50,60.00
7529,2023-01-18 23:36:43.927,646f045c53130a08d1d4014a,2.301191e+14,A01-018,사출기(18호),N1570-7SH,신M6CF1 FINGER CONTROL 사출품(신형)[FCD600],2023-01-19 오전 8:36:40,60.5,62.0,55.0,57.9,16.0,1,기포,55.0,58.850,57.175,59.20,60.875,62.0,-0.508376,-1.334271,58.42,59.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420428,2023-06-14 10:00:17.279,64898fb2851b289853a03fbd,2.306142e+14,A01-018,사출기(18호),N1217-8SH,M6AR1 BODY DETEND 사출품[SCR420],2023-06-14 오후 7:00:16,60.4,60.9,57.4,56.0,1030.0,0,NaN,56.0,58.675,57.050,58.90,60.525,60.9,-0.256967,-4.127447,58.00,59.80
420429,2023-06-14 10:01:27.290,64898ff7851b289853a03fbe,2.306142e+14,A01-018,사출기(18호),N1217-8SH,M6AR1 BODY DETEND 사출품[SCR420],2023-06-14 오후 7:01:24,60.5,61.6,57.4,56.0,1032.0,0,NaN,56.0,58.875,57.050,58.95,60.775,61.6,-0.091271,-3.858035,58.02,59.88
420430,2023-06-14 10:02:42.379,64899043851b289853a03fbf,2.306142e+14,A01-018,사출기(18호),N1217-8SH,M6AR1 BODY DETEND 사출품[SCR420],2023-06-14 오후 7:02:39,60.5,61.3,57.5,56.0,1034.0,0,NaN,56.0,58.825,57.125,59.00,60.700,61.3,-0.215227,-3.843592,58.10,59.90


In [15]:
#Final_Matched.to_csv('machine18.csv',index=True,encoding='cp949')

In [16]:
#Final_NonMatched.to_csv('machine18_Original.csv',index=True,encoding='cp949')